In [ ]:
import os
from googletrans import Translator, LANGUAGES
from gtts import gTTS
import pyttsx3
import json

PREFERENCES_FILE = "translation_preferences.json"

translator = Translator()

def load_preferences():
    if os.path.exists(PREFERENCES_FILE):
        with open(PREFERENCES_FILE, "r") as f:
            return json.load(f)
    return {"default_target_lang": None, "history": []}

def save_preferences(preferences):
    with open(PREFERENCES_FILE, "w") as f:
        json.dump(preferences, f, indent=4)

preferences = load_preferences()

def speak_text(text):
    try:
        tts = gTTS(text=text, lang='en')
        tts.save("translation.mp3")
        os.system("start translation.mp3" if os.name == 'nt' else "afplay translation.mp3")
    except Exception:
        engine = pyttsx3.init()
        engine.say(text)
        engine.runAndWait()

def translate_text():
    print("Welcome to the Advanced Language Translation Chatbot!")
    global preferences

    while True:
        input_text = input("\nEnter the text you want to translate (or type 'exit' to quit): ")
        if input_text.lower() == 'exit':
            print("Goodbye!")
            break

        detected_language = translator.detect(input_text).lang
        print(f"\nDetected source language: {LANGUAGES[detected_language]}")

        if not preferences["default_target_lang"]:
            print("\nSupported Languages:")
            for lang_code, lang_name in LANGUAGES.items():
                print(f"{lang_code}: {lang_name}")
            preferences["default_target_lang"] = input("\nEnter your preferred target language code (e.g., 'en' for English): ").lower()

        target_lang = preferences["default_target_lang"]

        if target_lang not in LANGUAGES:
            print("Invalid language code. Please try again.")
            preferences["default_target_lang"] = None
            continue

        translation = translator.translate(input_text, src=detected_language, dest=target_lang)
        print(f"\nTranslated text ({LANGUAGES[detected_language]} to {LANGUAGES[target_lang]}):")
        print(translation.text)

        preferences["history"].append({
            "source_text": input_text,
            "source_language": LANGUAGES[detected_language],
            "target_language": LANGUAGES[target_lang],
            "translated_text": translation.text
        })
        save_preferences(preferences)

        print("\nOptions:")
        print("1. Translate another text")
        print("2. Save translated text to file")
        print("3. Speak translated text")
        print("4. View translation history")
        print("5. Change target language")
        print("6. Exit")
        choice = input("Choose an option (1-6): ").strip()

        if choice == '2':
            with open("translation_output.txt", "a") as f:
                f.write(f"\n{LANGUAGES[detected_language]} to {LANGUAGES[target_lang]}: {translation.text}")
            print("Translation saved to 'translation_output.txt'.")
        elif choice == '3':
            speak_text(translation.text)
        elif choice == '4':
            print("\nTranslation History:")
            for idx, entry in enumerate(preferences["history"], start=1):
                print(f"{idx}. {entry['source_language']} -> {entry['target_language']}: {entry['translated_text']}")
        elif choice == '5':
            preferences["default_target_lang"] = input("\nEnter new target language code: ").lower()
        elif choice == '6':
            print("Goodbye!")
            break

translate_text()
